In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [3]:
from dataclasses import dataclass
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@dataclass
class RuntimeContext:
    db: SQLDatabase

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [4]:
SYSTEM = f"""You are a careful SQLite analyst of chinook database. Your name is cooper.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *."""

In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="moonshotai/kimi-k2-instruct-0905",
    model_provider="groq",
    temperature=1
)

In [6]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

config = {"configurable": {"thread_id": "1"}}

In [8]:
question = "Which table has the highest number of entries?"

response = agent.invoke(
    {"messages": question},
    context=RuntimeContext(db=db),
    config=config
    )
print(response["messages"][-1].content)

The table with the highest number of records in the chinook database is **PlaylistTrack** with **8,715 records**.

Here's the complete ranking of all tables by their record count:

1. **PlaylistTrack**: 8,715 records
2. **Track**: 3,503 records
3. **InvoiceLine**: 2,240 records
4. **Invoice**: 412 records
5. **Album**: 347 records
6. **Artist**: 275 records
7. **Customer**: 59 records
8. **Genre**: 25 records
9. **Playlist**: 18 records
10. **Employee**: 8 records
11. **MediaType**: 5 records
